In [571]:
import json
import os
import statistics
import pandas as pd
from pandas import json_normalize

In [572]:
with open('input_data/devices.json') as fh:
    devices = [json.loads(line) for line in fh.readlines()]

In [573]:
devicesData = pd.DataFrame.from_dict(devices, orient='columns')

In [574]:
devicesData.head()

,id,name,lat,lon
0,1001,zigbuu-17,36.928026,-2.687309
1,1002,zigbuu-22,40.723609,-5.204659
2,1003,zigbuu-52,35.772100,-78.638610
3,1004,zigbuu-124,42.210419,1.240819


In [575]:
stationsData = pd.read_csv (r'input_data/stations.csv')

In [576]:
stationsData.head()

,station_id,lat,lon,source,reports,country,measurement_reliability
0,022320-99999,64.433,15.633,isd,hourly,SE,1007.281089
1,023140-99999,62.100,13.250,isd,hourly,SE,822.825000
2,023160-99999,61.700,13.183,isd,hourly,SE,813.391100
3,024090-99999,59.050,12.700,isd,hourly,SE,749.935000
4,026270-99999,55.717,13.217,isd,hourly,SE,736.411589


In [577]:
dailySummary = pd.DataFrame(columns = ['device_id', 'device_name', 'lat', 'lon', 'date', 'avg_temp', 'meassurement_reliability_score', 'pm10', 'pm25'])

In [578]:
rootdir = 'scripts/input_data/weather'

for subdir, dirs, files in os.walk(rootdir):
    for file in files:
        
        with open(os.path.join(subdir, file), 'r') as fh:
            weather = [json.loads(line) for line in fh.readlines()]

        weatherData = json_normalize(weather, 'data', ['lat', 'lon', 'station_id'], record_prefix='data_')

        weatherData = weatherData[['station_id','lon', 'lat', 'data_temp', 'data_datetime']]
        
        for (columnName, columnData) in devicesData.iteritems():
            if columnName == 'lon':
                lon = columnData.values
                for x in range (0, len(lon)):
                    avgTemp = weatherData.loc[weatherData['lon'] == lon[x], 'data_temp'].mean(axis=0)

                    weatherData.loc[weatherData['lon'] == lon[x], 'data_datetime']
                    date = weatherData.loc[weatherData['lon'] == lon[x], 'data_datetime'].iloc[0]
                    date = date.replace(":00", "")

                    weatherData.loc[weatherData['lon'] == lon[x], 'station_id']
                    # check if there is two station values returned and get the harmonic mean of the two resulting rely values
                    if weatherData.loc[weatherData['lon'] == lon[x], 'station_id'].iloc[1] is None:
                        stationId = weatherData.loc[weatherData['lon'] == lon[x], 'station_id'].iloc[0]
                    else:
                        stationId1 = weatherData.loc[weatherData['lon'] == lon[x], 'station_id'].iloc[0]
                        stationId2 = weatherData.loc[weatherData['lon'] == lon[x], 'station_id'].iloc[1]
                        
                        stationsData.loc[stationsData['station_id'] == stationId1, 'measurement_reliability']
                        rely1 = stationsData.loc[stationsData['station_id'] == stationId1, 'measurement_reliability'].iloc[0]
                        
                        stationsData.loc[stationsData['station_id'] == stationId2, 'measurement_reliability']
                        rely2 = stationsData.loc[stationsData['station_id'] == stationId2, 'measurement_reliability'].iloc[0]
                        
                        data = [rely1, rely2]
                        
                        rely = statistics.harmonic_mean(data)

                    stationsData.loc[stationsData['station_id'] == stationId, 'measurement_reliability']
                    rely = stationsData.loc[stationsData['station_id'] == stationId, 'measurement_reliability'].iloc[0]

                    devicesData.loc[devicesData['lon'] == lon[x], 'id']
                    deviceId = devicesData.loc[devicesData['lon'] == lon[x], 'id'].iloc[0]

                    devicesData.loc[devicesData['lon'] == lon[x], 'name']
                    deviceName = devicesData.loc[devicesData['lon'] == lon[x], 'name'].iloc[0]

                    devicesData.loc[devicesData['lon'] == lon[x], 'lat']
                    lat = devicesData.loc[devicesData['lon'] == lon[x], 'lat'].iloc[0]
                    
                    with open('scripts/air_data/data.json', 'r') as fh:
                        air = [json.loads(line) for line in fh.readlines()]

                    airData = json_normalize(air, 'data', ['lat', 'lon'], record_prefix='data_')

                    airData = airData[['lat', 'lon', 'data_pm10','data_pm25', 'data_datetime']]

                    airData['data_datetime'] = airData.apply(lambda x: x['data_datetime'][:-3], axis = 1)

                    dates = airData['data_datetime'].unique()
                    
                    match = False
                    
                    for val in dates:
                        
                        if val == date: 
                            
                            avgPM10 = airData.loc[(airData['lon'] == round(lon[x], 2)) & (airData['data_datetime'] == val), 'data_pm10'].mean(axis=0)
                            avgPM25 = airData.loc[(airData['lon'] == round(lon[x], 2)) & (airData['data_datetime'] == val), 'data_pm25'].mean(axis=0)
                            dailySummary = dailySummary.append({'device_id': deviceId, 'device_name': deviceName, 'lat': lat, 'lon': lon[x], 'date': date, 'avg_temp': avgTemp, 'meassurement_reliability_score': rely, 'pm10': avgPM10, 'pm25': avgPM25}, ignore_index=True)
                            match = True  
                            
                    if match == False:
                        
                        dailySummary = dailySummary.append({'device_id': deviceId, 'device_name': deviceName, 'lat': lat, 'lon': lon[x], 'date': date, 'avg_temp': avgTemp, 'meassurement_reliability_score': rely}, ignore_index=True)

In [579]:
 dailySummary.tail(15)

,device_id,device_name,lat,lon,date,avg_temp,meassurement_reliability_score,pm10,pm25
105,1002,zigbuu-22,40.723609,-5.204659,2020-08-06,26.733333,36.5874,NaN,NaN
106,1003,zigbuu-52,35.772100,-78.638610,2020-08-06,25.450000,36.5874,NaN,NaN
107,1004,zigbuu-124,42.210419,1.240819,2020-08-06,23.875000,36.5874,NaN,NaN
108,1001,zigbuu-17,36.928026,-2.687309,2020-08-07,28.966667,36.5874,12.268930,3.448922
109,1002,zigbuu-22,40.723609,-5.204659,2020-08-07,27.400000,36.5874,45.292950,11.044711
110,1003,zigbuu-52,35.772100,-78.638610,2020-08-07,25.625000,36.5874,1.916219,0.968291
111,1004,zigbuu-124,42.210419,1.240819,2020-08-07,26.537500,36.5874,7.323290,2.491452
112,1001,zigbuu-17,36.928026,-2.687309,2020-08-08,28.425000,36.5874,19.712083,4.227155
113,1002,zigbuu-22,40.723609,-5.204659,2020-08-08,26.358333,36.5874,49.775121,15.768629
114,1003,zigbuu-52,35.772100,-78.638610,2020-08-08,26.125000,36.5874,2.513984,1.651116
